# Project Management

### Param

In [52]:
_pathTasks='tasks.csv'
_pathAvail='avail.csv'

## Internals

In [135]:
import pandas as pd
import datetime as dt
import numpy as np

### Load data

In [139]:
_TaskHeaders = ['TaskId','Name','WorkLoad','Who','StartDate','EndDate','Priority','PreviousTaskId']

def LoadTask(path):
        data = pd.read_csv(path, header=0,names=_TaskHeaders, dtype={'PreviousTaskId':np.int32})
        data.index=data.TaskId
        return data
def LoadAvailability(path):
    '''
    This function load an availability matrix 
    header : who, dates+
    It expects first column to be who, the other columns availabity per day in terme of days
    '''
    availCSV = pd.read_csv(path)
    datesHeaders = availCSV.columns[1:]    
    cols = ['Who']
    cols.append([dt.datetime.strptime(date, '%d/%m/%Y').date() for date in datesHeaders])
    availCSV.columns =cols
    return availCSV
    

### Save

In [ ]:
def SaveData(data,path):
    data.to_csv(path)

### Domain
* Find tasks of a personn
* Order tasks by priority then ID (just in case)
* Compute end dates
    * Find previous task of person and previous task id
    * Based on the max start date and workload, iterate to see the day when the task will be finised


In [122]:
def FindPreviousTaskOfSomeone(tasks, current):
    '''
    Find the previous task in order of doing
    Expects an order tasks pandas by priority desc and taskid 
    '''
    whosTasks = FindTasksOfSomeone(tasks,current.Who)    
    prevTasks= whosTasks.loc[:current.TaskId]
    if len(prevTasks)<2 :
        return None
    else :
        return prevTasks.iloc[-2]
    
    

In [61]:
def FindTasksOfSomeone(tasks, who):
    return tasks[tasks.Who==who]
def FindTaskIdById(tasks, taskid):
    return tasks[tasks.TaskId=taskid].head()
def OrderTaskByPriority(tasks):
    return tasks.sort_values(by=['Priority','TaskId'], ascending=[False,True])

## Load Data

In [140]:
_tasks = LoadTask(_pathTasks)

ValueError: Integer column has NA values in column 7

In [54]:
_avail = LoadAvailability(_pathAvail)

In [131]:
_tasks.head(10)

,TaskId,Name,WorkLoad,Who,StartDate,EndDate,Priority,PreviousTaskId
TaskId,,,,,,,,
1,1,Test 1,4,CGA,NaN,NaN,150,NaN
2,2,Test 2,4,CGA,NaN,NaN,130,3.0
3,3,Test 3,4,ACH,NaN,NaN,140,NaN


In [56]:
_avail.head(1)

,Who,"(2016-05-01,)"
0,'CGA',1


In [132]:
help (pd.read_csv)

Help on function read_csv in module pandas.io.parsers:

read_csv(filepath_or_buffer, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal='.', lineterminator=None, quotechar='"', quoting=0, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=False, error_bad_lines=True, warn_bad_lines=True, skip_footer=0, doublequote=True, delim_whitespace=False, as_recarray=False, compact_ints=False, use_unsigned=False, low_memory=True, buffer_lines=None, memory_map=False, float_precision=